# 함수 및 라이브러리 기본 세팅

In [399]:
import pandas as pd
import requests
from bs4  import BeautifulSoup
from time import sleep


from selenium                          import webdriver
from webdriver_manager.chrome          import ChromeDriverManager
from selenium.webdriver.support.select import Select
from selenium.common.exceptions        import NoSuchElementException

In [769]:
def get_players_data_by_game(players):
    """
    players: 요소가 선수명으로 이루어진 리스트
    
    이 함수는 statiz에서 각 선수가 뛴 모든 국내 시즌에서의 기록들을
    연도 단위로 일별로 가져옵니다.
    """
    all_players = {}

    driver  = webdriver.Chrome(executable_path=ChromeDriverManager().install())
    exception_players = {
        "김현수": "http://www.statiz.co.kr/player.php?opt=3&name=김현수&birth=1988-01-12",
        "김상수": "http://www.statiz.co.kr/player.php?opt=3&name=김상수&birth=1990-03-23",
        "박병호": "http://www.statiz.co.kr/player.php?opt=3&name=박병호&birth=1986-07-10",
        "박건우": "http://www.statiz.co.kr/player.php?opt=3&name=박건우&birth=1990-09-08",
    }
    for name in players:
        if name in exception_players:
            url = exception_players[name]
            driver.get(url)
            sleep(1)
        else:
            url = f'http://www.statiz.co.kr/player.php?opt=3&name={name}'
            driver.get(url)
            sleep(1)

        # 팀 이름, 선수 이름, 포지션
        player_info     = driver.find_element_by_class_name('callout').text
        player_team     = player_info.split('\n')[-1].split(',')[0]
        player_position = player_info.split(', ')[1]
            
        print(player_team, name, player_position)
        
        # 투수는 타자 기록이 없기 때문에 넘어간다.
        if player_position == '투수':
            print(f"{name}은(는) 투수입니다 !")
            continue

        seasons_wrapper = driver.find_element_by_xpath('/html/body/div[1]/div[1]/div/section[2]/div/div[2]/div/div[2]/div')
        seasons = seasons_wrapper.find_elements_by_tag_name('a')
        
        # 과거부터 최신 순으로
        years = [season.text for season in seasons]
        years.sort(reverse=False)
        
        player_data = {}
        for year in years:
            driver.get(url + f"&year={year}")
            sleep(1)

            html       = BeautifulSoup(driver.page_source, 'lxml')
            table_html = html.find_all("table")[2]
            table      = pd.read_html(str(table_html), encoding='cp949')[0]

            # table 정보 추가 및 기본 전처리
            table["이름"]  = name
            table["포지션"] = player_position
            table["팀"]    = player_team
            table = table.rename(columns={table.columns[0]: "경기일"})
            
            # 경기일 column에서 이상한 row 삭제 및 index 재정렬
            drop_index = table["경기일"][table["경기일"].apply(lambda x: x.find('월')) > 0].index
            table = table.drop(drop_index).reset_index(drop=True)
            table["경기일"] = table["경기일"].apply(lambda x: f"{year}-{x}")
            table["경기일"] = table["경기일"].astype('datetime64')
            table["연도"]  = table["경기일"].dt.year
            
            # 필요없는 column drop
            drop_cols = ["경기일", "포지션", "상대"]
            table = table.drop(drop_cols, axis=1)
            
            # 결과에 대해 전처리
            table["결과"] = table["결과"].apply(lambda x: 1 if x[0] == "W" else 0)

            player_data[year] = table

        all_players[name] = player_data

    driver.quit()
    return all_players


def group_by_day(df, day):
    group_day = int(len(df) / day)
    
    bundle_by_day = pd.DataFrame()
    for i in range(group_day):
        start_idx = day * i
        end_idx   = day * (i + 1)
        temp_df = df.loc[start_idx:end_idx, :]
        
        int_cols = [
            '결과', '선발', '타수', '득점', '안타', '2타', '3타', '홈런', '루타', '타점',
            '도루', '도실', '볼넷', '사구', '고4', '삼진', '병살', '희타', '희비'
        ]
        float_cols = [
            '타율', '출루', '장타', 'OPS', 
            '투구', 'avLI', 'RE24', 'WPA'
        ]
        
        # int col과 float col 연산
        # 하나의 row로 만든다.
        temp_df[int_cols]   = temp_df[int_cols].astype('int')
        temp_df[float_cols] = temp_df[float_cols].astype('float')
        
        
        temp_df.loc[start_idx, int_cols]   = temp_df[int_cols].sum()
        temp_df.loc[start_idx, float_cols] = temp_df[float_cols].mean()
        temp_df.loc[start_idx, "P"]        = temp_df["P"].value_counts().sort_values().index[-1]
        temp_df.loc[start_idx, "타순"]      = temp_df["타순"].value_counts().sort_values().index[-1]
        
        bundle_by_day = bundle_by_day.append(temp_df.loc[start_idx, :])
        
    return bundle_by_day.reset_index(drop=True)


def get_player_data_a_year(all_player_data, players, year, division=8):
    """
    all_player_data : 선수 이름을 key 값으로 가지고 있으며, 그 안에 year와, dataframe을 key와 value로 가지고 있는 dictionary
    players         : 가져오고자 하는 선수명의 리스트
    year            : 가져오고자 하는 데이터의 연도
    division        : 경기묶음의 단위
    선수의 특정 연도 데이터를 division 수의 경기만큼 묶어 반환합니다.
    """
    all_df_by_day = pd.DataFrame()
    for player in players:
        if year in all_player_data[player]:
            df        = all_player_data[player][year]
            df_by_day = group_by_day(df, division)
            all_df_by_day = all_df_by_day.append(df_by_day).reset_index(drop=True)

    return all_df_by_day

# 데이터 가져오기

## 프리미어 15 선수 시즌 데이터 가져오기

In [767]:
for player in premier_player_kbo_data:
    if "2015" in premier_player_kbo_data[player]:
        print(1)
    else:
        print(player)

1
1
1
1
1
이대호
1
1
1


In [762]:
premier_15      = pd.read_csv("data/15_premier.csv")
main_idx        = premier_15["타수"].sort_values(ascending=False)[:9].index
premier_15      = premier_15.loc[main_idx].reset_index(drop=True)
premier_players = premier_15["이름"].tolist()
premier_year    = "2015"

# 크롤링 가져오기
premier_player_kbo_data = get_players_data_by_game(premier_players)
premier_player_kbo_data = get_player_data_a_year(premier_player_kbo_data, premier_players, premier_year, division=8)

# 백업 df 생성
backup_premier = premier_player_kbo_data.copy()



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [/Users/orangemush/.wdm/drivers/chromedriver/mac64/92.0.4515.43/chromedriver] found in cache


LG 정근우 2루수
LG 김현수 지명타자
키움 박병호 1루수
KT 황재균 3루수
키움 이용규 좌익수
롯데 이대호 지명타자
두산 김재호 유격수
NC 양의지 지명타자
삼성 강민호 포수


/opt/homebrew/Caskroom/miniforge/base/envs/nano/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
/opt/homebrew/Caskroom/miniforge/base/envs/nano/lib/python3.8/site-packages/pandas/core/indexing.py:1797: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, v, pi)
/opt/homebrew/Caskroom/miniforge/base/envs/nano/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is tr

KeyError: '2015'

## 19연도 kbo, premier 데이터 가져오기

In [723]:
pd.options.display.max_columns = 999

premier_2019    = pd.read_csv("data/19_premier.csv")
sub_idx         = premier_2019["타수"].sort_values()[:4].index
premier_2019    = premier_2019.drop(sub_idx, axis=0).reset_index(drop=True)
premier_players = premier_2019["이름"].tolist()
premier_year    = "2019"

# 크롤링 가져오기
premier_player_kbo_data = get_players_data_by_game(premier_players)
premier_player_kbo_data = get_player_data_a_year(premier_player_kbo_data, premier_players, premier_year, division=8)

# 백업 df 생성
backup_premier = premier_player_kbo_data.copy()



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Get LATEST driver version for 92.0.4515
Trying to download new driver from https://chromedriver.storage.googleapis.com/92.0.4515.43/chromedriver_mac64.zip
Driver has been saved in cache [/Users/orangemush/.wdm/drivers/chromedriver/mac64/92.0.4515.43]


두산 김재환 좌익수
삼성 김상수 2루수
키움 김하성 유격수
LG 김현수 지명타자
롯데 민병헌 중견수
NC 박민우 2루수
키움 박병호 1루수
NC 양의지 지명타자
키움 이정후 중견수


## 21년도 kbo 시즌 데이터 가져오기

In [737]:
# 투수 포지션을 제외한 도쿄 올림픽 멤버
tokyo_year    = "2021"
tokyo_players = [
    "박건우", "김현수", "이정후", "박해민", "강백호",
    "황재균", "오지환", "김혜성", "양의지", "강민호",
    "오재일", "박민우", "최주환", "허경민"
]

# 크롤링 가져오기
tokyo_player_kbo_data = get_players_data_by_game(tokyo_players)
tokyo_player_kbo_data = get_player_data_a_year(tokyo_player_kbo_data, tokyo_players, tokyo_year, division=8)

# 백업 df 생성
backup_tokyo = tokyo_player_kbo_data.copy()



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [/Users/orangemush/.wdm/drivers/chromedriver/mac64/92.0.4515.43/chromedriver] found in cache


두산 박건우 중견수
LG 김현수 지명타자
키움 이정후 중견수
삼성 박해민 중견수
KT 강백호 1루수
KT 황재균 3루수
LG 오지환 유격수
키움 김혜성 유격수
NC 양의지 지명타자
삼성 강민호 포수
삼성 오재일 1루수
NC 박민우 2루수
SSG 최주환 2루수
두산 허경민 3루수


/opt/homebrew/Caskroom/miniforge/base/envs/nano/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
/opt/homebrew/Caskroom/miniforge/base/envs/nano/lib/python3.8/site-packages/pandas/core/indexing.py:1797: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, v, pi)
/opt/homebrew/Caskroom/miniforge/base/envs/nano/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is tr

# 변수명 변경

In [774]:
premier_player_kbo_data = backup_premier.copy()
# tokyo_player_kbo_data   = backup_tokyo.copy()

columns = [
    'RES', 'ORD', 'P', 'SP', 'AB', 'R', 'H', '2B', '3B', 'HR',
    'TB', 'RBI', 'SB', 'CS', 'BB', 'HBP', 'IBB', 'SO', 'GDP',
    'SH', 'SF', 'AVG', 'OBP', 'SLG', 'OPS', 'PN',
    'avLI', 'RE24', 'WPA', 'PLAYER', 'TEAM', 'YEAR'
]

# 컬럼명 변경
premier_player_kbo_data.columns = columns
# tokyo_player_kbo_data.columns   = columns

# tokyo_player_kbo_data.head(3)
premier_player_kbo_data.head(3)

,RES,ORD,P,SP,AB,R,H,2B,3B,HR,TB,RBI,SB,CS,BB,HBP,IBB,SO,GDP,SH,SF,AVG,OBP,SLG,OPS,PN,avLI,RE24,WPA,PLAYER,TEAM,YEAR
0,7.0,2,2B,9.0,27.0,3.0,5.0,2.0,0.0,1.0,10.0,2.0,2.0,0.0,4.0,0.0,0.0,4.0,2.0,5.0,0.0,0.088667,0.193778,0.163111,0.357111,12.222222,0.866667,-0.248889,-0.023444,정근우,LG,2015.0
1,4.0,2,2B,9.0,37.0,9.0,10.0,3.0,0.0,2.0,19.0,10.0,1.0,0.0,3.0,0.0,0.0,2.0,3.0,1.0,0.0,0.222333,0.322111,0.417889,0.740222,15.444444,1.116667,-0.126667,0.011556,정근우,LG,2015.0
2,4.0,3,2B,9.0,34.0,6.0,9.0,2.0,0.0,0.0,11.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0,0.218222,0.292111,0.369556,0.661667,14.111111,1.324444,-0.596667,-0.032222,정근우,LG,2015.0


In [787]:
# print(len(tokyo_player_kbo_data.columns))
# len(premier_player_kbo_data.columns)

37


36

In [775]:
def add_default_derived_var(df):
    df['B']    = df['H'] - (df['2B'] + df['3B'] + df['HR'])
    df['GPA']  = (df.OBP * 1.8 + df.SLG) / 4
    df['BB/K'] = df.BB / df.SO
    df['ISOP'] = df.SLG - df.AVG
    df['XBH']  = df['2B'] + df['3B'] + df.HR


# 기본 파생변수 추가
add_default_derived_var(premier_player_kbo_data)
# add_default_derived_var(tokyo_player_kbo_data)
    
# 실수 변수들 가져오기
float_cols = premier_player_kbo_data.dtypes[premier_player_kbo_data.dtypes == "float"].index.tolist()
# float_cols = tokyo_player_kbo_data.dtypes[tokyo_player_kbo_data.dtypes == "float"].index.tolist()

# 실수 변수들 3째 자리까지 표기
premier_player_kbo_data[float_cols] = premier_player_kbo_data[float_cols].apply(round, args=(3, ))
# tokyo_player_kbo_data[float_cols]   = tokyo_player_kbo_data[float_cols].apply(round, args=(3, ))

premier_player_kbo_data.head(3)
# tokyo_player_kbo_data.head(3)

,RES,ORD,P,SP,AB,R,H,2B,3B,HR,TB,RBI,SB,CS,BB,HBP,IBB,SO,GDP,SH,SF,AVG,OBP,SLG,OPS,PN,avLI,RE24,WPA,PLAYER,TEAM,YEAR,B,GPA,BB/K,ISOP,XBH
0,7.0,2,2B,9.0,27.0,3.0,5.0,2.0,0.0,1.0,10.0,2.0,2.0,0.0,4.0,0.0,0.0,4.0,2.0,5.0,0.0,0.089,0.194,0.163,0.357,12.222,0.867,-0.249,-0.023,정근우,LG,2015.0,2.0,0.128,1.0,0.074,3.0
1,4.0,2,2B,9.0,37.0,9.0,10.0,3.0,0.0,2.0,19.0,10.0,1.0,0.0,3.0,0.0,0.0,2.0,3.0,1.0,0.0,0.222,0.322,0.418,0.740,15.444,1.117,-0.127,0.012,정근우,LG,2015.0,5.0,0.249,1.5,0.196,5.0
2,4.0,3,2B,9.0,34.0,6.0,9.0,2.0,0.0,0.0,11.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0,0.218,0.292,0.370,0.662,14.111,1.324,-0.597,-0.032,정근우,LG,2015.0,7.0,0.224,1.0,0.151,2.0


In [776]:
a = [
    'PLAYER', 'TEAM', 'YEAR', 'RES', 'ORD', 'P', 'SP', 'AB', 'R', 
    'H', 'B', '2B', '3B', 'HR', 'XBH', 'TB', 'RBI',
    'SB', 'CS', 'BB', 'HBP', 'IBB', 'SO', 'GDP', 'SH', 'SF', 'AVG', 'OBP',
    'SLG', 'ISOP', 'OPS','GPA','PN', 'avLI', 'RE24', 'WPA'
]
premier_player_kbo_data = premier_player_kbo_data[a]
# tokyo_player_kbo_data   = tokyo_player_kbo_data[a]

# csv 파일 저장

In [783]:
premier_15_path = "data/2015_premier_player_season.csv"
premier_19_path = "data/2019_premier_player_season.csv"
# tokyo_path   = "data/2021_tokyo_player_season.csv"

premier_player_kbo_data.to_csv(premier_15_path, index=False)
# premier_player_kbo_data.to_csv(premier_19_path, index=False)
# tokyo_player_kbo_data.to_csv(tokyo_path, index=False)

In [781]:
pd.read_csv(premier_15_path)

,PLAYER,TEAM,YEAR,RES,ORD,P,SP,AB,R,H,B,2B,3B,HR,XBH,TB,RBI,SB,CS,BB,HBP,IBB,SO,GDP,SH,SF,AVG,OBP,SLG,ISOP,OPS,GPA,PN,avLI,RE24,WPA
0,정근우,LG,2015.0,7.0,2,2B,9.0,27.0,3.0,5.0,2.0,2.0,0.0,1.0,3.0,10.0,2.0,2.0,0.0,4.0,0.0,0.0,4.0,2.0,5.0,0.0,0.089,0.194,0.163,0.074,0.357,0.128,12.222,0.867,-0.249,-0.023
1,정근우,LG,2015.0,4.0,2,2B,9.0,37.0,9.0,10.0,5.0,3.0,0.0,2.0,5.0,19.0,10.0,1.0,0.0,3.0,0.0,0.0,2.0,3.0,1.0,0.0,0.222,0.322,0.418,0.196,0.740,0.249,15.444,1.117,-0.127,0.012
2,정근우,LG,2015.0,4.0,3,2B,9.0,34.0,6.0,9.0,7.0,2.0,0.0,0.0,2.0,11.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0,0.218,0.292,0.370,0.151,0.662,0.224,14.111,1.324,-0.597,-0.032
3,정근우,LG,2015.0,6.0,3,2B,9.0,32.0,8.0,6.0,4.0,2.0,0.0,0.0,2.0,8.0,2.0,1.0,1.0,5.0,0.0,0.0,3.0,2.0,0.0,0.0,0.227,0.307,0.362,0.134,0.668,0.229,13.444,1.048,-0.314,-0.006
4,정근우,LG,2015.0,5.0,3,2B,9.0,32.0,4.0,10.0,8.0,2.0,0.0,0.0,2.0,12.0,7.0,3.0,2.0,6.0,0.0,0.0,5.0,0.0,1.0,0.0,0.229,0.314,0.339,0.110,0.653,0.226,16.333,1.274,0.447,0.095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,강민호,삼성,2015.0,3.0,6,C,9.0,27.0,4.0,10.0,6.0,2.0,0.0,2.0,4.0,18.0,6.0,0.0,0.0,9.0,1.0,0.0,3.0,2.0,0.0,0.0,0.303,0.422,0.637,0.334,1.059,0.349,17.000,1.289,0.324,0.121
125,강민호,삼성,2015.0,6.0,6,C,8.0,25.0,4.0,7.0,4.0,0.0,0.0,3.0,3.0,16.0,5.0,0.0,0.0,4.0,1.0,0.0,9.0,2.0,0.0,0.0,0.310,0.433,0.650,0.340,1.083,0.358,14.222,0.977,0.070,0.076
126,강민호,삼성,2015.0,4.0,6,C,8.0,29.0,2.0,9.0,5.0,4.0,0.0,0.0,4.0,13.0,3.0,0.0,0.0,3.0,1.0,0.0,11.0,1.0,0.0,1.0,0.311,0.430,0.637,0.327,1.067,0.353,15.444,1.079,0.008,0.008
127,강민호,삼성,2015.0,5.0,6,C,8.0,27.0,3.0,10.0,7.0,1.0,0.0,2.0,3.0,17.0,5.0,0.0,0.0,2.0,0.0,0.0,7.0,0.0,0.0,1.0,0.310,0.426,0.626,0.316,1.052,0.348,12.778,0.833,0.177,0.013


In [784]:
pd.read_csv(premier_19_path)

,PLAYER,TEAM,YEAR,RES,ORD,P,SP,AB,R,H,B,2B,3B,HR,XBH,TB,RBI,SB,CS,BB,HBP,IBB,SO,GDP,SH,SF,AVG,OBP,SLG,ISOP,OPS,GPA,PN,avLI,RE24,WPA
0,김재환,두산,2019.0,7.0,4,LF,9.0,32.0,5.0,9.0,5.0,1.0,0.0,3.0,4.0,19.0,12.0,0.0,0.0,5.0,0.0,0.0,12.0,2.0,0.0,1.0,0.211,0.407,0.450,0.239,0.857,0.296,16.778,1.134,0.471,0.040
1,김재환,두산,2019.0,5.0,4,LF,9.0,32.0,3.0,9.0,7.0,1.0,1.0,0.0,2.0,12.0,6.0,0.0,0.0,5.0,0.0,0.0,6.0,0.0,0.0,2.0,0.266,0.361,0.524,0.258,0.885,0.293,15.333,0.859,0.007,0.007
2,김재환,두산,2019.0,6.0,4,LF,9.0,32.0,9.0,10.0,7.0,1.0,0.0,2.0,3.0,17.0,9.0,1.0,0.0,8.0,0.0,0.0,7.0,1.0,0.0,1.0,0.281,0.370,0.469,0.189,0.839,0.284,16.778,0.954,0.603,0.052
3,김재환,두산,2019.0,5.0,4,LF,9.0,36.0,9.0,14.0,9.0,2.0,0.0,3.0,5.0,25.0,7.0,0.0,0.0,5.0,0.0,1.0,9.0,1.0,0.0,0.0,0.305,0.396,0.546,0.241,0.942,0.315,15.778,0.769,0.544,0.055
4,김재환,두산,2019.0,6.0,4,LF,9.0,36.0,4.0,11.0,10.0,0.0,1.0,0.0,1.0,13.0,7.0,0.0,1.0,1.0,0.0,0.0,10.0,1.0,0.0,1.0,0.313,0.392,0.529,0.215,0.920,0.309,13.778,1.039,-0.087,0.015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,이정후,키움,2019.0,5.0,1,LF,9.0,39.0,8.0,15.0,11.0,2.0,1.0,1.0,4.0,22.0,8.0,1.0,1.0,2.0,1.0,0.0,2.0,0.0,0.0,1.0,0.331,0.380,0.461,0.130,0.841,0.286,17.111,1.019,0.338,0.005
140,이정후,키움,2019.0,4.0,3,CF,9.0,41.0,8.0,10.0,8.0,2.0,0.0,0.0,2.0,12.0,3.0,0.0,0.0,4.0,0.0,0.0,2.0,2.0,0.0,0.0,0.331,0.380,0.462,0.131,0.842,0.286,18.333,1.062,-0.461,-0.074
141,이정후,키움,2019.0,6.0,3,CF,9.0,36.0,5.0,12.0,9.0,3.0,0.0,0.0,3.0,15.0,1.0,0.0,1.0,3.0,0.0,0.0,3.0,3.0,0.0,0.0,0.326,0.376,0.452,0.126,0.829,0.282,17.111,1.211,-0.417,-0.037
142,이정후,키움,2019.0,6.0,3,CF,9.0,36.0,8.0,17.0,16.0,1.0,0.0,0.0,1.0,18.0,4.0,0.0,0.0,5.0,0.0,0.0,1.0,2.0,0.0,0.0,0.337,0.388,0.461,0.124,0.849,0.290,18.222,0.974,0.361,0.036


In [785]:
pd.read_csv(tokyo_path)

,RES,ORD,P,SP,AB,R,H,2B,3B,HR,TB,RBI,SB,CS,BB,HBP,IBB,SO,GDP,SH,SF,AVG,OBP,SLG,OPS,PN,avLI,RE24,WPA,PLAYER,TEAM,YEAR,B,GPA,BB/K,ISOP,XBH
0,5.0,3,RF,9.0,35.0,7.0,16.0,4.0,0.0,2.0,26.0,7.0,1.0,0.0,3.0,0.0,0.0,4.0,1.0,0.0,0.0,0.358,0.371,0.796,1.168,16.222,1.039,0.587,0.069,박건우,두산,2021.0,10.0,0.366,0.750,0.438,6.0
1,5.0,3,RF,9.0,31.0,8.0,11.0,1.0,0.0,0.0,12.0,4.0,0.0,0.0,5.0,0.0,0.0,4.0,1.0,0.0,0.0,0.421,0.471,0.633,1.103,14.000,0.961,-0.048,0.015,박건우,두산,2021.0,10.0,0.370,1.250,0.212,1.0
2,5.0,3,CF,9.0,25.0,1.0,5.0,1.0,0.0,0.0,6.0,5.0,0.0,0.0,5.0,2.0,0.0,3.0,3.0,0.0,2.0,0.382,0.454,0.523,0.977,14.778,1.081,-0.377,-0.060,박건우,두산,2021.0,4.0,0.335,1.667,0.141,1.0
3,5.0,3,CF,8.0,36.0,7.0,14.0,4.0,0.0,0.0,18.0,6.0,1.0,0.0,1.0,1.0,0.0,6.0,1.0,0.0,0.0,0.366,0.433,0.494,0.927,15.333,1.016,0.268,-0.000,박건우,두산,2021.0,10.0,0.318,0.167,0.128,4.0
4,3.0,3,CF,9.0,32.0,4.0,6.0,0.0,0.0,0.0,6.0,4.0,1.0,0.0,5.0,0.0,0.0,5.0,1.0,0.0,0.0,0.351,0.419,0.463,0.882,15.889,1.106,0.057,-0.006,박건우,두산,2021.0,6.0,0.304,1.000,0.112,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,4.0,1,3B,6.0,28.0,4.0,11.0,2.0,0.0,1.0,16.0,4.0,0.0,1.0,2.0,1.0,0.0,0.0,2.0,0.0,1.0,0.344,0.400,0.421,0.821,12.667,1.172,-0.136,-0.010,허경민,두산,2021.0,8.0,0.285,inf,0.077,3.0
108,3.0,1,3B,9.0,40.0,2.0,9.0,1.0,0.0,1.0,13.0,5.0,1.0,0.0,1.0,0.0,0.0,1.0,3.0,0.0,1.0,0.345,0.395,0.444,0.840,17.000,1.008,-0.496,0.002,허경민,두산,2021.0,7.0,0.289,1.000,0.099,2.0
109,5.0,1,3B,8.0,36.0,5.0,10.0,2.0,0.0,1.0,15.0,4.0,0.0,2.0,2.0,1.0,0.0,3.0,3.0,0.0,0.0,0.326,0.372,0.427,0.799,15.333,0.873,-0.204,0.025,허경민,두산,2021.0,7.0,0.274,0.667,0.101,3.0
110,4.0,1,3B,9.0,38.0,7.0,12.0,4.0,1.0,0.0,18.0,6.0,1.0,1.0,5.0,0.0,0.0,2.0,3.0,0.0,0.0,0.325,0.376,0.432,0.808,17.778,1.063,0.090,0.014,허경민,두산,2021.0,7.0,0.277,2.500,0.107,5.0
